# The Value of Cultural Similarity for Predicting Migration


## Carolina Coimbra Vieira, Sophie Lohmann, Emilio Zagheni

In [1]:
shhh <- suppressPackageStartupMessages # It's a library, so shhh!

shhh(library(tidyr))
shhh(library(dplyr))
shhh(library(readr))
shhh(library(lsa))
shhh(library(philentropy))
shhh(library('fastDummies'))

Warning message:
"package 'tidyr' was built under R version 4.1.3"
Warning message:
"package 'tidyr' was built under R version 4.1.3"
Warning message:
"package 'dplyr' was built under R version 4.1.3"
Warning message:
"package 'dplyr' was built under R version 4.1.3"
Warning message:
"package 'readr' was built under R version 4.1.3"
Warning message:
"package 'readr' was built under R version 4.1.3"
Warning message:
"package 'lsa' was built under R version 4.1.3"
Warning message:
"package 'lsa' was built under R version 4.1.3"
Warning message:
"package 'SnowballC' was built under R version 4.1.1"
Warning message:
"package 'SnowballC' was built under R version 4.1.1"
Warning message:
"package 'philentropy' was built under R version 4.1.3"
Warning message:
"package 'philentropy' was built under R version 4.1.3"


## Population 2019
### Datasource: [UN](https://population.un.org/wpp/Download/Standard/Population/)

In [2]:
pop_2019 <- read.csv("data/subset-pop-un2019.csv")

head(pop_2019)

,country,population
,<chr>,<int>
1,Argentina,44780675
2,Australia,25203200
3,Brazil,211049519
4,Chile,18952035
5,France,65129731
6,Great Britain,67530161


,country,population
,<chr>,<int>
1,Argentina,44780675
2,Australia,25203200
3,Brazil,211049519
4,Chile,18952035
5,France,65129731
6,Great Britain,67530161


In [3]:
countries <- c(pop_2019$country)

countries

[1] "Argentina"     "Australia"     "Brazil"        "Chile"        
 [5] "France"        "Great Britain" "Indonesia"     "Japan"        
 [9] "Malaysia"      "Mexico"        "Russia"        "Singapore"    
[13] "South Korea"   "Spain"         "Turkey"        "United States"

[1] "Argentina"     "Australia"     "Brazil"        "Chile"        
 [5] "France"        "Great Britain" "Indonesia"     "Japan"        
 [9] "Malaysia"      "Mexico"        "Russia"        "Singapore"    
[13] "South Korea"   "Spain"         "Turkey"        "United States"

## Geographic info: area  
### Datasource: [CEPII](http://www.cepii.fr/CEPII/en/bdd_modele/presentation.asp?id=6)

<b>Country-level variables</b>   
    *`iso2, iso3, cnum`*: ISO codes in two and three characters, and in three numbers respectively.  
    *`country, pays`*: Name of country in English and French respectively.   
    **`area`: Country’s area in $km^2$.**   
    *`dis_int`*: Internal distance of country i, $d_{ii} = .67 \sqrt{area/\pi}$ (an often used measure of average distance between producers and consumers in a country, see Head and Mayer, 2002 for more on this topic).  
    *`landlocked`*: Dummy variable set equal to 1 for landlocked countries.    
    *`continent`*: Continent to which the country is belonging.   
    *`langoff_i`*: Official or national languages and languages spoken by at least 20% of the population of the country (and spoken in another country of the world) following the same logic than the "open-circuit languages" in Mélitz (2002).  
    *`lang20_i`*: Languages (mother tongue, lingua francas or second languages) spoken by at least 20% of the population of the country.  
    *`lang9_i`*: Languages (mother tongue, lingua francas or second languages) spoken by between 9% and 20% of the population of the country.  
    *`colonizeri`*: Colonizers of the country for a relatively long period of time and with a substantial participation in the governance of the colonized country.  
    *`short_colonizeri`*: Colonizers of the country for a relatively short period of time or with only low involvement in the governance of the colonized country.
    
<b>Cities variables used in the computation of distances</b>  
    *`city_en, city_fr`*: Names of capitals or main cities of the country in English and French.   
    *`lat, lon`*: Latitude and longitude of the city.  
    *`cap`*: Variable equals to 1 if the city is the capital of the country, to 0 if the city is the most populated city (maincity equals to 1) but not the capital, and to 2 in the cases of two capitals, if the city is the most populated but the “second” capital or the previous capital.  
    *`maincity`*: Variable coded as 1 when the city is the most populated of the country and as 2 otherwise.  
    *`citynum`*: Number of cities for each country used to calculate our weighted distances described in the next section.  

In [4]:
geo <- read.csv("data/cepii_geo2.csv")

head(geo)

,iso2,iso3,cnum,country,pays,area,dis_int,landlocked,continent,city_en,...,lang9_2,lang9_3,lang9_4,colonizer1,colonizer2,colonizer3,colonizer4,short_colonizer1,short_colonizer2,short_colonizer3
,<chr>,<chr>,<int>,<chr>,<chr>,<int>,<dbl>,<int>,<chr>,<chr>,...,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,AW,ABW,533,Aruba,Aruba,193,5.225315,0,America,Oranjestad,...,.,.,.,NLD,.,.,.,.,.,.
2,AF,AFG,4,Afghanistan,Afghanistan,652225,303.761400,1,Asia,Kabul,...,Uzbek,.,.,.,.,.,.,GBR,.,.
3,AO,AGO,24,Angola,Angola,1246700,419.966600,0,Africa,Luanda,...,.,.,.,PRT,.,.,.,.,.,.
4,AI,AIA,660,Anguilla,Anguilla,102,3.798690,0,America,The Valley,...,.,.,.,GBR,.,.,.,.,.,.
5,AL,ALB,8,Albania,Albanie,28748,63.773110,0,Europe,Tirana,...,.,.,.,TUR,.,.,.,.,.,.
6,AD,AND,20,Andorra,Andorre,453,8.005398,0,Europe,Andorra la Vella,...,.,.,.,.,.,.,.,.,.,.


,iso2,iso3,cnum,country,pays,area,dis_int,landlocked,continent,city_en,...,lang9_2,lang9_3,lang9_4,colonizer1,colonizer2,colonizer3,colonizer4,short_colonizer1,short_colonizer2,short_colonizer3
,<chr>,<chr>,<int>,<chr>,<chr>,<int>,<dbl>,<int>,<chr>,<chr>,...,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,AW,ABW,533,Aruba,Aruba,193,5.225315,0,America,Oranjestad,...,.,.,.,NLD,.,.,.,.,.,.
2,AF,AFG,4,Afghanistan,Afghanistan,652225,303.761400,1,Asia,Kabul,...,Uzbek,.,.,.,.,.,.,GBR,.,.
3,AO,AGO,24,Angola,Angola,1246700,419.966600,0,Africa,Luanda,...,.,.,.,PRT,.,.,.,.,.,.
4,AI,AIA,660,Anguilla,Anguilla,102,3.798690,0,America,The Valley,...,.,.,.,GBR,.,.,.,.,.,.
5,AL,ALB,8,Albania,Albanie,28748,63.773110,0,Europe,Tirana,...,.,.,.,TUR,.,.,.,.,.,.
6,AD,AND,20,Andorra,Andorre,453,8.005398,0,Europe,Andorra la Vella,...,.,.,.,.,.,.,.,.,.,.


In [5]:
colnames(geo)

[1] "iso2"             "iso3"             "cnum"             "country"         
 [5] "pays"             "area"             "dis_int"          "landlocked"      
 [9] "continent"        "city_en"          "city_fr"          "lat"             
[13] "lon"              "cap"              "maincity"         "citynum"         
[17] "langoff_1"        "langoff_2"        "langoff_3"        "lang20_1"        
[21] "lang20_2"         "lang20_3"         "lang20_4"         "lang9_1"         
[25] "lang9_2"          "lang9_3"          "lang9_4"          "colonizer1"      
[29] "colonizer2"       "colonizer3"       "colonizer4"       "short_colonizer1"
[33] "short_colonizer2" "short_colonizer3"

[1] "iso2"             "iso3"             "cnum"             "country"         
 [5] "pays"             "area"             "dis_int"          "landlocked"      
 [9] "continent"        "city_en"          "city_fr"          "lat"             
[13] "lon"              "cap"              "maincity"         "citynum"         
[17] "langoff_1"        "langoff_2"        "langoff_3"        "lang20_1"        
[21] "lang20_2"         "lang20_3"         "lang20_4"         "lang9_1"         
[25] "lang9_2"          "lang9_3"          "lang9_4"          "colonizer1"      
[29] "colonizer2"       "colonizer3"       "colonizer4"       "short_colonizer1"
[33] "short_colonizer2" "short_colonizer3"

In [6]:
area <- geo %>% filter (country %in% countries) %>%
                select(iso2, iso3, country, area) %>% 
                unique()

area

,iso2,iso3,country,area
,<chr>,<chr>,<chr>,<int>
1,AR,ARG,Argentina,2766889
2,AU,AUS,Australia,7686848
4,BR,BRA,Brazil,8511920
6,CL,CHL,Chile,756945
7,ES,ESP,Spain,505954
8,FR,FRA,France,547026
9,GB,GBR,Great Britain,244110
10,ID,IDN,Indonesia,1933658
11,JP,JPN,Japan,377801


,iso2,iso3,country,area
,<chr>,<chr>,<chr>,<int>
1,AR,ARG,Argentina,2766889
2,AU,AUS,Australia,7686848
4,BR,BRA,Brazil,8511920
6,CL,CHL,Chile,756945
7,ES,ESP,Spain,505954
8,FR,FRA,France,547026
9,GB,GBR,Great Britain,244110
10,ID,IDN,Indonesia,1933658
11,JP,JPN,Japan,377801


#### Population + Area

In [7]:
area_pop <- merge(area, pop_2019)

area_pop

country,iso2,iso3,area,population
<chr>,<chr>,<chr>,<int>,<int>
Argentina,AR,ARG,2766889,44780675
Australia,AU,AUS,7686848,25203200
Brazil,BR,BRA,8511920,211049519
Chile,CL,CHL,756945,18952035
France,FR,FRA,547026,65129731
Great Britain,GB,GBR,244110,67530161
Indonesia,ID,IDN,1933658,270625567
Japan,JP,JPN,377801,126860299
Malaysia,MY,MYS,329758,31949789


country,iso2,iso3,area,population
<chr>,<chr>,<chr>,<int>,<int>
Argentina,AR,ARG,2766889,44780675
Australia,AU,AUS,7686848,25203200
Brazil,BR,BRA,8511920,211049519
Chile,CL,CHL,756945,18952035
France,FR,FRA,547026,65129731
Great Britain,GB,GBR,244110,67530161
Indonesia,ID,IDN,1933658,270625567
Japan,JP,JPN,377801,126860299
Malaysia,MY,MYS,329758,31949789


In [8]:
isos3 <- area$iso3

isos3

[1] "ARG" "AUS" "BRA" "CHL" "ESP" "FRA" "GBR" "IDN" "JPN" "KOR" "MEX" "MYS"
[13] "RUS" "SGP" "TUR" "USA"

[1] "ARG" "AUS" "BRA" "CHL" "ESP" "FRA" "GBR" "IDN" "JPN" "KOR" "MEX" "MYS"
[13] "RUS" "SGP" "TUR" "USA"

## Geographic distance
### Datasource: [CEPII](http://www.cepii.fr/CEPII/en/bdd_modele/presentation.asp?id=6)  

<b>Variables</b>  
    *`iso_o, iso_d`*: ISO codes in three characters, refering to the country of origin and destination, respectively.  
    *`dist`*: Geodesic distances from lat/long of most populous cities (simple distance).   
    *`distcap`*: geodesic distance between capital cities (simple distance).    
    *`distw`*: population weighted distance, theta = 1 (weighted distance).   
    *`distwces`*: population weighted distance, theta = -1, which corresponds to the usual coefficient estimated from gravity models of bilateral trade flows (weighted distance).  
    *`contig`*: dummy variable indicating whether the two countries are contiguous (share a land border).  
    *`comlang_off, comlang_ethno`*: dummy variable indicating whether the two countries share a common language. There are two common languages dummies, the first one based on the fact that two countries share a common official language, and the other one set to one if a language is spoken by at least 9% of the population in both countries.     
    *`colony`*: dummy variable indicating whether the two countries have ever had a colonial link.  
    *`comcol`*: dummy variable indicating whether the two countries have had a common colonizer after 1945.  
    *`curcol`*: dummy variable indicating whether the two countries are currently in a colonial relationship.  
    *`col45`*: dummy variable indicating whether the two countries have had a colonial relationship -a-f-t-e-r- 1945 (share common colonizer pre 1945).  
    *`smctry`*: dummy variable indicating whether the two countries were/are the same country.  

**[TO DO]: check Maciej distance**   
<b>Maciej</b>  
    *`dist_pop_weighted`*: population-weighted average distance between biggest cities  
    *`dist_biggest_cities`*: average distance between biggest cities  
^ most similar to distwces  
    *`dist_unweighted`*: average distance between (?) (not population weighted)  

In [9]:
dists <- read.csv("data/cepii_dist2.csv")

head(dists)

,iso_o,iso_d,contig,comlang_off,comlang_ethno,colony,comcol,curcol,col45,smctry,dist,distcap,distw,distwces
,<chr>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<chr>,<chr>
1,ABW,ABW,0,0,0,0,0,0,0,0,5.225315,5.225315,25.09354,23.04723
2,ABW,AFG,0,0,0,0,0,0,0,0,13257.810000,13257.810000,13168.22,13166.37
3,ABW,AGO,0,0,0,0,0,0,0,0,9516.913000,9516.913000,9587.316,9584.193
4,ABW,AIA,0,0,1,0,0,0,0,0,983.268200,983.268200,976.8974,976.8916
5,ABW,ALB,0,0,0,0,0,0,0,0,9091.742000,9091.742000,9091.576,9091.466
6,ABW,AND,0,1,0,0,0,0,0,0,7572.788000,7572.788000,7570.084,7570.083


,iso_o,iso_d,contig,comlang_off,comlang_ethno,colony,comcol,curcol,col45,smctry,dist,distcap,distw,distwces
,<chr>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<chr>,<chr>
1,ABW,ABW,0,0,0,0,0,0,0,0,5.225315,5.225315,25.09354,23.04723
2,ABW,AFG,0,0,0,0,0,0,0,0,13257.810000,13257.810000,13168.22,13166.37
3,ABW,AGO,0,0,0,0,0,0,0,0,9516.913000,9516.913000,9587.316,9584.193
4,ABW,AIA,0,0,1,0,0,0,0,0,983.268200,983.268200,976.8974,976.8916
5,ABW,ALB,0,0,0,0,0,0,0,0,9091.742000,9091.742000,9091.576,9091.466
6,ABW,AND,0,1,0,0,0,0,0,0,7572.788000,7572.788000,7570.084,7570.083


**New variable:**  
    *`shared_hist`*: dummy variable indicating whether the two countries have: *`colony, comcol, curcol, col45, smctry`*

In [10]:
dists <- dists %>% mutate(shared_hist = ifelse(colony + comcol + curcol + col45 + smctry, 1, 0), .keep = "unused")

head(dists)

,iso_o,iso_d,contig,comlang_off,comlang_ethno,dist,distcap,distw,distwces,shared_hist
,<chr>,<chr>,<int>,<int>,<int>,<dbl>,<dbl>,<chr>,<chr>,<dbl>
1,ABW,ABW,0,0,0,5.225315,5.225315,25.09354,23.04723,0
2,ABW,AFG,0,0,0,13257.810000,13257.810000,13168.22,13166.37,0
3,ABW,AGO,0,0,0,9516.913000,9516.913000,9587.316,9584.193,0
4,ABW,AIA,0,0,1,983.268200,983.268200,976.8974,976.8916,0
5,ABW,ALB,0,0,0,9091.742000,9091.742000,9091.576,9091.466,0
6,ABW,AND,0,1,0,7572.788000,7572.788000,7570.084,7570.083,0


,iso_o,iso_d,contig,comlang_off,comlang_ethno,dist,distcap,distw,distwces,shared_hist
,<chr>,<chr>,<int>,<int>,<int>,<dbl>,<dbl>,<chr>,<chr>,<dbl>
1,ABW,ABW,0,0,0,5.225315,5.225315,25.09354,23.04723,0
2,ABW,AFG,0,0,0,13257.810000,13257.810000,13168.22,13166.37,0
3,ABW,AGO,0,0,0,9516.913000,9516.913000,9587.316,9584.193,0
4,ABW,AIA,0,0,1,983.268200,983.268200,976.8974,976.8916,0
5,ABW,ALB,0,0,0,9091.742000,9091.742000,9091.576,9091.466,0
6,ABW,AND,0,1,0,7572.788000,7572.788000,7570.084,7570.083,0


#### Population + Area + Geographic distance  + Shared history

In [11]:
area_pop_gd_d <- area_pop %>% select(!iso2) %>%
                               merge(dists, by.x=c("iso3"), by.y=c("iso_d")) %>%
                               rename(iso_d = iso3, destination = country, area_d = area, pop_d = population) 

area_pop_gd_o <- area_pop %>% select(!iso2) %>% 
                               merge(dists, by.x=c("iso3"), by.y=c("iso_o")) %>%
                               rename(iso_o = iso3, origin = country, area_o = area, pop_o = population) 

area_pop_gdist <- merge(area_pop_gd_d, area_pop_gd_o)

head(area_pop_gdist)

,iso_d,iso_o,contig,comlang_off,comlang_ethno,dist,distcap,distw,distwces,shared_hist,destination,area_d,pop_d,origin,area_o,pop_o
,<chr>,<chr>,<int>,<int>,<int>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<int>,<int>,<chr>,<int>,<int>
1,ARG,ARG,0,0,0,625.6475,625.6475,533.9082,96.14934,0,Argentina,2766889,44780675,Argentina,2766889,44780675
2,ARG,AUS,0,0,0,11801.3600,11733.8800,12044.57,12018.27,0,Argentina,2766889,44780675,Australia,7686848,25203200
3,ARG,BRA,1,0,0,1691.0670,2353.2570,2391.846,2089.281,0,Argentina,2766889,44780675,Brazil,8511920,211049519
4,ARG,CHL,1,1,1,1128.3170,1128.3170,1156.726,941.6383,0,Argentina,2766889,44780675,Chile,756945,18952035
5,ARG,ESP,0,1,1,10065.8500,10065.8500,10079.66,10055.9,1,Argentina,2766889,44780675,Spain,505954,46736782
6,ARG,FRA,0,0,0,11072.2500,11072.2500,10932.34,10921.86,0,Argentina,2766889,44780675,France,547026,65129731


,iso_d,iso_o,contig,comlang_off,comlang_ethno,dist,distcap,distw,distwces,shared_hist,destination,area_d,pop_d,origin,area_o,pop_o
,<chr>,<chr>,<int>,<int>,<int>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<int>,<int>,<chr>,<int>,<int>
1,ARG,ARG,0,0,0,625.6475,625.6475,533.9082,96.14934,0,Argentina,2766889,44780675,Argentina,2766889,44780675
2,ARG,AUS,0,0,0,11801.3600,11733.8800,12044.57,12018.27,0,Argentina,2766889,44780675,Australia,7686848,25203200
3,ARG,BRA,1,0,0,1691.0670,2353.2570,2391.846,2089.281,0,Argentina,2766889,44780675,Brazil,8511920,211049519
4,ARG,CHL,1,1,1,1128.3170,1128.3170,1156.726,941.6383,0,Argentina,2766889,44780675,Chile,756945,18952035
5,ARG,ESP,0,1,1,10065.8500,10065.8500,10079.66,10055.9,1,Argentina,2766889,44780675,Spain,505954,46736782
6,ARG,FRA,0,0,0,11072.2500,11072.2500,10932.34,10921.86,0,Argentina,2766889,44780675,France,547026,65129731


## Language  
### Datasource: [CEPII](http://www.cepii.fr/CEPII/en/bdd_modele/presentation.asp?id=19)   

<b>Variables [Sarah's notes]</b>  
    *`col`*: common official language (0 or 1); 19 languages considered.    
    *`csl`*: p(two random people understand a common language) >= *`cnl`*.  
    *`cnl`*: p(two random people share a native language).  
    *`lp`*: lexical closeness of native langauges; set to 0 when *`cnl`* is 1 or 0 also set to 0 if there is no dominant native language (e.g. India).  
    *`lp1`*: tree based. 4 possibilities, 2 languages belonging to:  
        0: separate family trees  
        0.25: different branches of same tree (English and French),  
        0.50: the same branch (English and German),  
        0.75: the same sub-branch (German and Dutch)  
    *`lp2`*: lexical similarity of 200 words, continuous scale 0-100 normalized *`lp1`*, *`lp2`* so coefficients are comparable to each other and *`col, prox1, prox2`* are unadjusted versions of *`lp1`* and *`lp2`*?  
  

In [12]:
langs <- read.csv("data/cepii_language.csv")

head(langs)

,X,iso_o,country_o,iso_d,country_d,col,csl,cnl,prox1,lp1,prox2,lp2,cl,cle
,<int>,<chr>,<chr>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,AFG,Afghanistan,ALB,Albania,0,0,0,0.25,NA,0.07342058,NA,NA,0.08662836
2,2,AFG,Afghanistan,DZA,Algeria,0,0,0,0.00,0.000000,0.08527792,0.7640970,0.09992715,0.10061873
3,3,AFG,Afghanistan,AND,Andorra,0,0,0,0.25,1.945866,0.12025990,1.0775385,0.14091842,0.14189370
4,4,AFG,Afghanistan,AGO,Angola,0,0,0,0.25,NA,0.11890529,NA,NA,0.14029542
5,5,AFG,Afghanistan,AIA,Anguilla,0,0,0,0.25,NA,0.09291404,NA,NA,0.10962854
6,6,AFG,Afghanistan,ATG,Antigua and Barbuda,0,0,0,0.25,1.945866,0.09291404,0.8325174,0.10887503,0.10962854


,X,iso_o,country_o,iso_d,country_d,col,csl,cnl,prox1,lp1,prox2,lp2,cl,cle
,<int>,<chr>,<chr>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,AFG,Afghanistan,ALB,Albania,0,0,0,0.25,NA,0.07342058,NA,NA,0.08662836
2,2,AFG,Afghanistan,DZA,Algeria,0,0,0,0.00,0.000000,0.08527792,0.7640970,0.09992715,0.10061873
3,3,AFG,Afghanistan,AND,Andorra,0,0,0,0.25,1.945866,0.12025990,1.0775385,0.14091842,0.14189370
4,4,AFG,Afghanistan,AGO,Angola,0,0,0,0.25,NA,0.11890529,NA,NA,0.14029542
5,5,AFG,Afghanistan,AIA,Anguilla,0,0,0,0.25,NA,0.09291404,NA,NA,0.10962854
6,6,AFG,Afghanistan,ATG,Antigua and Barbuda,0,0,0,0.25,1.945866,0.09291404,0.8325174,0.10887503,0.10962854


In [13]:
langs <- langs %>% filter (iso_o %in% isos3 & iso_d %in% isos3)

head(langs)

,X,iso_o,country_o,iso_d,country_d,col,csl,cnl,prox1,lp1,prox2,lp2,cl,cle
,<int>,<chr>,<chr>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1368,ARG,Argentina,AUS,Australia,0,0.02058416,0.00080000,0.25,1.945866,0.13052547,1.1695188,0.15362507,0.15468276
2,1383,ARG,Argentina,BRA,Brazil,0,0.06052872,0.00080000,0.75,5.837599,0.42439765,3.8026376,0.49770382,0.50114280
3,1396,ARG,Argentina,CHL,Chile,1,0.98010004,0.85439998,0.00,0.000000,0.00000000,0.0000000,0.87344128,0.87256539
4,1422,ARG,Argentina,FRA,France,0,0.13392781,0.01039232,0.50,3.891733,0.25450581,2.2803929,0.30551848,0.30756098
5,1441,ARG,Argentina,IDN,Indonesia,0,0.00000000,0.00000000,0.00,0.000000,0.07436915,0.6663537,0.08714446,0.08774757
6,1448,ARG,Argentina,JPN,Japan,0,0.00000000,0.00000000,0.00,0.000000,0.06896222,0.6179070,0.08080870,0.08136797


,X,iso_o,country_o,iso_d,country_d,col,csl,cnl,prox1,lp1,prox2,lp2,cl,cle
,<int>,<chr>,<chr>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1368,ARG,Argentina,AUS,Australia,0,0.02058416,0.00080000,0.25,1.945866,0.13052547,1.1695188,0.15362507,0.15468276
2,1383,ARG,Argentina,BRA,Brazil,0,0.06052872,0.00080000,0.75,5.837599,0.42439765,3.8026376,0.49770382,0.50114280
3,1396,ARG,Argentina,CHL,Chile,1,0.98010004,0.85439998,0.00,0.000000,0.00000000,0.0000000,0.87344128,0.87256539
4,1422,ARG,Argentina,FRA,France,0,0.13392781,0.01039232,0.50,3.891733,0.25450581,2.2803929,0.30551848,0.30756098
5,1441,ARG,Argentina,IDN,Indonesia,0,0.00000000,0.00000000,0.00,0.000000,0.07436915,0.6663537,0.08714446,0.08774757
6,1448,ARG,Argentina,JPN,Japan,0,0.00000000,0.00000000,0.00,0.000000,0.06896222,0.6179070,0.08080870,0.08136797


#### Population + Area + Geographic distance + Shared history +  Language

In [14]:
area_pop_gdist_lang <- langs %>% select(!c(X, country_o, country_d)) %>%
                                merge(area_pop_gdist)

head(area_pop_gdist_lang)

,iso_o,iso_d,col,csl,cnl,prox1,lp1,prox2,lp2,cl,...,distcap,distw,distwces,shared_hist,destination,area_d,pop_d,origin,area_o,pop_o
,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<chr>,<chr>,<dbl>,<chr>,<int>,<int>,<chr>,<int>,<int>
1,ARG,AUS,0,0.02058416,0.00080000,0.25,1.945866,0.1305255,1.169519,0.1536251,...,11733.880,12044.57,12018.27,0,Australia,7686848,25203200,Argentina,2766889,44780675
2,ARG,BRA,0,0.06052872,0.00080000,0.75,5.837599,0.4243976,3.802638,0.4977038,...,2353.257,2391.846,2089.281,0,Brazil,8511920,211049519,Argentina,2766889,44780675
3,ARG,CHL,1,0.98010004,0.85439998,0.00,0.000000,0.0000000,0.000000,0.8734413,...,1128.317,1156.726,941.6383,0,Chile,756945,18952035,Argentina,2766889,44780675
4,ARG,ESP,1,0.98013985,0.85445821,0.00,0.000000,0.0000000,0.000000,0.8734919,...,10065.850,10079.66,10055.9,1,Spain,505954,46736782,Argentina,2766889,44780675
5,ARG,FRA,0,0.13392781,0.01039232,0.50,3.891733,0.2545058,2.280393,0.3055185,...,11072.250,10932.34,10921.86,0,France,547026,65129731,Argentina,2766889,44780675
6,ARG,GBR,0,0.08325152,0.00000000,0.25,1.945866,0.1305255,1.169519,0.1529474,...,11147.480,11136.96,11128.22,0,Great Britain,244110,67530161,Argentina,2766889,44780675


,iso_o,iso_d,col,csl,cnl,prox1,lp1,prox2,lp2,cl,...,distcap,distw,distwces,shared_hist,destination,area_d,pop_d,origin,area_o,pop_o
,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<chr>,<chr>,<dbl>,<chr>,<int>,<int>,<chr>,<int>,<int>
1,ARG,AUS,0,0.02058416,0.00080000,0.25,1.945866,0.1305255,1.169519,0.1536251,...,11733.880,12044.57,12018.27,0,Australia,7686848,25203200,Argentina,2766889,44780675
2,ARG,BRA,0,0.06052872,0.00080000,0.75,5.837599,0.4243976,3.802638,0.4977038,...,2353.257,2391.846,2089.281,0,Brazil,8511920,211049519,Argentina,2766889,44780675
3,ARG,CHL,1,0.98010004,0.85439998,0.00,0.000000,0.0000000,0.000000,0.8734413,...,1128.317,1156.726,941.6383,0,Chile,756945,18952035,Argentina,2766889,44780675
4,ARG,ESP,1,0.98013985,0.85445821,0.00,0.000000,0.0000000,0.000000,0.8734919,...,10065.850,10079.66,10055.9,1,Spain,505954,46736782,Argentina,2766889,44780675
5,ARG,FRA,0,0.13392781,0.01039232,0.50,3.891733,0.2545058,2.280393,0.3055185,...,11072.250,10932.34,10921.86,0,France,547026,65129731,Argentina,2766889,44780675
6,ARG,GBR,0,0.08325152,0.00000000,0.25,1.945866,0.1305255,1.169519,0.1529474,...,11147.480,11136.96,11128.22,0,Great Britain,244110,67530161,Argentina,2766889,44780675


In [15]:
dim(area_pop_gdist_lang)

[1] 240  25

[1] 240  25

## GDP (constant 2010) 2019  
### Datasource: [World Bank](https://databank.worldbank.org/source/world-development-indicators)

*`GDP`*: GDP (constant 2010) in 2019 in US$

GDP at purchaser's prices is the sum of gross value added by all resident producers in the economy plus any product taxes and minus any subsidies not included in the value of the products. It is calculated without making deductions for depreciation of fabricated assets or for depletion and degradation of natural resources. Data are in constant 2010 U.S. dollars. Dollar figures for GDP are converted from domestic currencies using 2010 official exchange rates. For a few countries where the official exchange rate does not reflect the rate effectively applied to actual foreign exchange transactions, an alternative conversion factor is used.

In [16]:
gdp <- read.csv("data/subset-GDP_constant2010-wb2019.csv")

head(gdp)

,iso_country,GDP
,<chr>,<dbl>
1,AUS,1.450499e+12
2,ARG,4.378134e+11
3,BRA,2.364446e+12
4,CHL,2.850370e+11
5,FRA,2.971919e+12
6,IDN,1.204457e+12


,iso_country,GDP
,<chr>,<dbl>
1,AUS,1.450499e+12
2,ARG,4.378134e+11
3,BRA,2.364446e+12
4,CHL,2.850370e+11
5,FRA,2.971919e+12
6,IDN,1.204457e+12


In [17]:
area_pop_gdist_lang_gdp <- area_pop_gdist_lang %>% 
                               merge(gdp, by.x=c("iso_o"), by.y=c("iso_country")) %>% 
                               rename(GDP_o = GDP) %>%
                               merge(gdp, by.x=c("iso_d"), by.y=c("iso_country")) %>%
                               rename(GDP_d = GDP)
                               
head(area_pop_gdist_lang_gdp)

,iso_d,iso_o,col,csl,cnl,prox1,lp1,prox2,lp2,cl,...,distwces,shared_hist,destination,area_d,pop_d,origin,area_o,pop_o,GDP_o,GDP_d
,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<chr>,<dbl>,<chr>,<int>,<int>,<chr>,<int>,<int>,<dbl>,<dbl>
1,ARG,CHL,1,0.98010004,0.8544,0.00,0.000000,0.0000000,0.000000,0.8734413,...,941.6383,0,Argentina,2766889,44780675,Chile,756945,18952035,2.850370e+11,437813398163
2,ARG,GBR,0,0.08325152,0.0000,0.25,1.945866,0.1305255,1.169519,0.1529474,...,11128.22,0,Argentina,2766889,44780675,Great Britain,244110,67530161,2.913557e+12,437813398163
3,ARG,AUS,0,0.02058416,0.0008,0.25,1.945866,0.1305255,1.169519,0.1536251,...,12018.27,0,Argentina,2766889,44780675,Australia,7686848,25203200,1.450499e+12,437813398163
4,ARG,KOR,0,0.00000000,0.0000,0.00,0.000000,0.0000000,0.000000,0.0000000,...,19137.75,0,Argentina,2766889,44780675,South Korea,99484,51225321,1.482760e+12,437813398163
5,ARG,BRA,0,0.06052872,0.0008,0.75,5.837599,0.4243976,3.802638,0.4977038,...,2089.281,0,Argentina,2766889,44780675,Brazil,8511920,211049519,2.364446e+12,437813398163
6,ARG,MEX,1,0.98010004,0.8832,0.00,0.000000,0.0000000,0.000000,0.8984749,...,7460.572,0,Argentina,2766889,44780675,Mexico,1967210,127575529,1.309882e+12,437813398163


,iso_d,iso_o,col,csl,cnl,prox1,lp1,prox2,lp2,cl,...,distwces,shared_hist,destination,area_d,pop_d,origin,area_o,pop_o,GDP_o,GDP_d
,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<chr>,<dbl>,<chr>,<int>,<int>,<chr>,<int>,<int>,<dbl>,<dbl>
1,ARG,CHL,1,0.98010004,0.8544,0.00,0.000000,0.0000000,0.000000,0.8734413,...,941.6383,0,Argentina,2766889,44780675,Chile,756945,18952035,2.850370e+11,437813398163
2,ARG,GBR,0,0.08325152,0.0000,0.25,1.945866,0.1305255,1.169519,0.1529474,...,11128.22,0,Argentina,2766889,44780675,Great Britain,244110,67530161,2.913557e+12,437813398163
3,ARG,AUS,0,0.02058416,0.0008,0.25,1.945866,0.1305255,1.169519,0.1536251,...,12018.27,0,Argentina,2766889,44780675,Australia,7686848,25203200,1.450499e+12,437813398163
4,ARG,KOR,0,0.00000000,0.0000,0.00,0.000000,0.0000000,0.000000,0.0000000,...,19137.75,0,Argentina,2766889,44780675,South Korea,99484,51225321,1.482760e+12,437813398163
5,ARG,BRA,0,0.06052872,0.0008,0.75,5.837599,0.4243976,3.802638,0.4977038,...,2089.281,0,Argentina,2766889,44780675,Brazil,8511920,211049519,2.364446e+12,437813398163
6,ARG,MEX,1,0.98010004,0.8832,0.00,0.000000,0.0000000,0.000000,0.8984749,...,7460.572,0,Argentina,2766889,44780675,Mexico,1967210,127575529,1.309882e+12,437813398163


In [18]:
dim(area_pop_gdist_lang_gdp)

[1] 240  27

[1] 240  27

In [19]:
gdp <- read.csv("data/subset-GDPpercapita_constant2010-wb2019.csv")

head(gdp)

,iso_country,GDP
,<chr>,<dbl>
1,ARG,12716.22
2,AUS,58868.48
3,BRA,8592.21
4,CHL,13761.37
5,ESP,28087.90
6,FRA,38832.02


,iso_country,GDP
,<chr>,<dbl>
1,ARG,12716.22
2,AUS,58868.48
3,BRA,8592.21
4,CHL,13761.37
5,ESP,28087.90
6,FRA,38832.02


In [20]:
area_pop_gdist_lang_gdp <- area_pop_gdist_lang_gdp %>% 
                               merge(gdp, by.x=c("iso_o"), by.y=c("iso_country")) %>% 
                               rename(GDP_percapita_o = GDP) %>%
                               merge(gdp, by.x=c("iso_d"), by.y=c("iso_country")) %>%
                               rename(GDP_percapita_d = GDP)
                               
head(area_pop_gdist_lang_gdp)

,iso_d,iso_o,col,csl,cnl,prox1,lp1,prox2,lp2,cl,...,destination,area_d,pop_d,origin,area_o,pop_o,GDP_o,GDP_d,GDP_percapita_o,GDP_percapita_d
,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<chr>,<int>,<int>,<chr>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
1,ARG,MYS,0,0.00000000,0.0000000,0.00,0.000000,0.08409883,0.7535324,0.09854551,...,Argentina,2766889,44780675,Malaysia,329758,31949789,3.989466e+11,437813398163,11114.543,12716.22
2,ARG,GBR,0,0.08325152,0.0000000,0.25,1.945866,0.13052547,1.1695188,0.15294743,...,Argentina,2766889,44780675,Great Britain,244110,67530161,2.913557e+12,437813398163,47491.595,12716.22
3,ARG,AUS,0,0.02058416,0.0008000,0.25,1.945866,0.13052547,1.1695188,0.15362507,...,Argentina,2766889,44780675,Australia,7686848,25203200,1.450499e+12,437813398163,58868.481,12716.22
4,ARG,JPN,0,0.00000000,0.0000000,0.00,0.000000,0.06896222,0.6179070,0.08080870,...,Argentina,2766889,44780675,Japan,377801,126860299,6.187014e+12,437813398163,36043.784,12716.22
5,ARG,ESP,1,0.98013985,0.8544582,0.00,0.000000,0.00000000,0.0000000,0.87349188,...,Argentina,2766889,44780675,Spain,505954,46736782,1.572013e+12,437813398163,28087.898,12716.22
6,ARG,IDN,0,0.00000000,0.0000000,0.00,0.000000,0.07436915,0.6663537,0.08714446,...,Argentina,2766889,44780675,Indonesia,1933658,270625567,1.204457e+12,437813398163,3892.422,12716.22


,iso_d,iso_o,col,csl,cnl,prox1,lp1,prox2,lp2,cl,...,destination,area_d,pop_d,origin,area_o,pop_o,GDP_o,GDP_d,GDP_percapita_o,GDP_percapita_d
,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<chr>,<int>,<int>,<chr>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
1,ARG,MYS,0,0.00000000,0.0000000,0.00,0.000000,0.08409883,0.7535324,0.09854551,...,Argentina,2766889,44780675,Malaysia,329758,31949789,3.989466e+11,437813398163,11114.543,12716.22
2,ARG,GBR,0,0.08325152,0.0000000,0.25,1.945866,0.13052547,1.1695188,0.15294743,...,Argentina,2766889,44780675,Great Britain,244110,67530161,2.913557e+12,437813398163,47491.595,12716.22
3,ARG,AUS,0,0.02058416,0.0008000,0.25,1.945866,0.13052547,1.1695188,0.15362507,...,Argentina,2766889,44780675,Australia,7686848,25203200,1.450499e+12,437813398163,58868.481,12716.22
4,ARG,JPN,0,0.00000000,0.0000000,0.00,0.000000,0.06896222,0.6179070,0.08080870,...,Argentina,2766889,44780675,Japan,377801,126860299,6.187014e+12,437813398163,36043.784,12716.22
5,ARG,ESP,1,0.98013985,0.8544582,0.00,0.000000,0.00000000,0.0000000,0.87349188,...,Argentina,2766889,44780675,Spain,505954,46736782,1.572013e+12,437813398163,28087.898,12716.22
6,ARG,IDN,0,0.00000000,0.0000000,0.00,0.000000,0.07436915,0.6663537,0.08714446,...,Argentina,2766889,44780675,Indonesia,1933658,270625567,1.204457e+12,437813398163,3892.422,12716.22


In [21]:
dim(area_pop_gdist_lang_gdp)

[1] 240  29

[1] 240  29

## Migration (stock) 2019  
### Datasource: [UN](https://www.un.org/en/development/desa/population/migration/data/estimates2/estimates19.asp)

The dataset presents estimates of international migrant by age, sex and origin. Estimates are presented for 1990, 1995, 2000, 2005, 2010, 2015 and 2019 and are available for all countries and areas of the world. The estimates are based on official statistics on the foreign-born or the foreign population.   
International migrant stock - Total international migrant stock   

In [22]:
migration_stock_wide <- read.csv("data/subset-stocks-migration-un2019.csv")
migration_stock_wide[is.na(migration_stock_wide)] <- 0

head(migration_stock_wide)

,destination,Total,Total16,Argentina,Australia,Brazil,Chile,France,Great.Britain,Indonesia,Japan,Malaysia,Mexico,Russia,Singapore,South.Korea,Spain,Turkey,United.States
,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,Argentina,2212879,397927,0,279,49647,216855,8303,959,21,4531,2,1475,1917,2,8296,99769,646,5225
2,Australia,7549270,2109825,17583,0,42552,33707,40733,1262204,87274,53175,174136,6760,28735,64739,121204,19594,42880,114549
3,Brazil,807006,229897,39078,791,0,20747,12851,6650,978,65955,163,3601,1865,154,11531,41321,486,23726
4,Chile,939992,177893,72813,3952,20370,0,12562,6668,343,3231,100,6887,2344,237,4247,22842,1019,20278
5,France,8334875,1110686,14253,9192,63208,15857,0,176672,5446,23536,2793,13851,70463,2512,22794,303245,327508,59356
6,Great Britain,9552110,1073094,12239,145692,47487,8668,170142,0,11543,52930,84638,10457,38917,58432,13171,98372,104491,215915


,destination,Total,Total16,Argentina,Australia,Brazil,Chile,France,Great.Britain,Indonesia,Japan,Malaysia,Mexico,Russia,Singapore,South.Korea,Spain,Turkey,United.States
,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,Argentina,2212879,397927,0,279,49647,216855,8303,959,21,4531,2,1475,1917,2,8296,99769,646,5225
2,Australia,7549270,2109825,17583,0,42552,33707,40733,1262204,87274,53175,174136,6760,28735,64739,121204,19594,42880,114549
3,Brazil,807006,229897,39078,791,0,20747,12851,6650,978,65955,163,3601,1865,154,11531,41321,486,23726
4,Chile,939992,177893,72813,3952,20370,0,12562,6668,343,3231,100,6887,2344,237,4247,22842,1019,20278
5,France,8334875,1110686,14253,9192,63208,15857,0,176672,5446,23536,2793,13851,70463,2512,22794,303245,327508,59356
6,Great Britain,9552110,1073094,12239,145692,47487,8668,170142,0,11543,52930,84638,10457,38917,58432,13171,98372,104491,215915


In [23]:
migration_stock <- migration_stock_wide %>% 
                pivot_longer(!c(destination, Total, Total16), names_to = "origin", values_to = "m_stock") %>%
                select(destination, Total, origin, m_stock) %>%
                rename(total_imm_in_d = Total)
migration_stock$origin <- chartr(".", " ", migration_stock$origin)

head(migration_stock)

destination,total_imm_in_d,origin,m_stock
<chr>,<int>,<chr>,<int>
Argentina,2212879,Argentina,0
Argentina,2212879,Australia,279
Argentina,2212879,Brazil,49647
Argentina,2212879,Chile,216855
Argentina,2212879,France,8303
Argentina,2212879,Great Britain,959


destination,total_imm_in_d,origin,m_stock
<chr>,<int>,<chr>,<int>
Argentina,2212879,Argentina,0
Argentina,2212879,Australia,279
Argentina,2212879,Brazil,49647
Argentina,2212879,Chile,216855
Argentina,2212879,France,8303
Argentina,2212879,Great Britain,959


In [24]:
area_pop_mig_d <- area_pop %>% select(country, iso3) %>% 
                               merge(migration_stock, by.x=c("country"), by.y=c("destination")) %>%
                               rename(iso_d = iso3, destination = country) 

area_pop_mig_o <- area_pop %>% select(country, iso3) %>% 
                               merge(migration_stock, by.x=c("country"), by.y=c("origin")) %>%
                               rename(iso_o = iso3, origin = country) 

migration_stock <- merge(area_pop_mig_d, area_pop_mig_o)

head(migration_stock)

,destination,total_imm_in_d,origin,m_stock,iso_d,iso_o
,<chr>,<int>,<chr>,<int>,<chr>,<chr>
1,Argentina,2212879,Argentina,0,ARG,ARG
2,Argentina,2212879,Australia,279,ARG,AUS
3,Argentina,2212879,Brazil,49647,ARG,BRA
4,Argentina,2212879,Chile,216855,ARG,CHL
5,Argentina,2212879,France,8303,ARG,FRA
6,Argentina,2212879,Great Britain,959,ARG,GBR


,destination,total_imm_in_d,origin,m_stock,iso_d,iso_o
,<chr>,<int>,<chr>,<int>,<chr>,<chr>
1,Argentina,2212879,Argentina,0,ARG,ARG
2,Argentina,2212879,Australia,279,ARG,AUS
3,Argentina,2212879,Brazil,49647,ARG,BRA
4,Argentina,2212879,Chile,216855,ARG,CHL
5,Argentina,2212879,France,8303,ARG,FRA
6,Argentina,2212879,Great Britain,959,ARG,GBR


In [25]:
official_dataset <- merge(area_pop_gdist_lang_gdp, migration_stock) %>% select(!total_imm_in_d)

head(official_dataset)

,iso_d,iso_o,destination,origin,col,csl,cnl,prox1,lp1,prox2,...,shared_hist,area_d,pop_d,area_o,pop_o,GDP_o,GDP_d,GDP_percapita_o,GDP_percapita_d,m_stock
,<chr>,<chr>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
1,ARG,AUS,Argentina,Australia,0,0.02058416,0.00080000,0.25,1.945866,0.1305255,...,0,2766889,44780675,7686848,25203200,1.450499e+12,437813398163,58868.48,12716.22,279
2,ARG,BRA,Argentina,Brazil,0,0.06052872,0.00080000,0.75,5.837599,0.4243976,...,0,2766889,44780675,8511920,211049519,2.364446e+12,437813398163,8592.21,12716.22,49647
3,ARG,CHL,Argentina,Chile,1,0.98010004,0.85439998,0.00,0.000000,0.0000000,...,0,2766889,44780675,756945,18952035,2.850370e+11,437813398163,13761.37,12716.22,216855
4,ARG,ESP,Argentina,Spain,1,0.98013985,0.85445821,0.00,0.000000,0.0000000,...,1,2766889,44780675,505954,46736782,1.572013e+12,437813398163,28087.90,12716.22,99769
5,ARG,FRA,Argentina,France,0,0.13392781,0.01039232,0.50,3.891733,0.2545058,...,0,2766889,44780675,547026,65129731,2.971919e+12,437813398163,38832.02,12716.22,8303
6,ARG,GBR,Argentina,Great Britain,0,0.08325152,0.00000000,0.25,1.945866,0.1305255,...,0,2766889,44780675,244110,67530161,2.913557e+12,437813398163,47491.59,12716.22,959


,iso_d,iso_o,destination,origin,col,csl,cnl,prox1,lp1,prox2,...,shared_hist,area_d,pop_d,area_o,pop_o,GDP_o,GDP_d,GDP_percapita_o,GDP_percapita_d,m_stock
,<chr>,<chr>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
1,ARG,AUS,Argentina,Australia,0,0.02058416,0.00080000,0.25,1.945866,0.1305255,...,0,2766889,44780675,7686848,25203200,1.450499e+12,437813398163,58868.48,12716.22,279
2,ARG,BRA,Argentina,Brazil,0,0.06052872,0.00080000,0.75,5.837599,0.4243976,...,0,2766889,44780675,8511920,211049519,2.364446e+12,437813398163,8592.21,12716.22,49647
3,ARG,CHL,Argentina,Chile,1,0.98010004,0.85439998,0.00,0.000000,0.0000000,...,0,2766889,44780675,756945,18952035,2.850370e+11,437813398163,13761.37,12716.22,216855
4,ARG,ESP,Argentina,Spain,1,0.98013985,0.85445821,0.00,0.000000,0.0000000,...,1,2766889,44780675,505954,46736782,1.572013e+12,437813398163,28087.90,12716.22,99769
5,ARG,FRA,Argentina,France,0,0.13392781,0.01039232,0.50,3.891733,0.2545058,...,0,2766889,44780675,547026,65129731,2.971919e+12,437813398163,38832.02,12716.22,8303
6,ARG,GBR,Argentina,Great Britain,0,0.08325152,0.00000000,0.25,1.945866,0.1305255,...,0,2766889,44780675,244110,67530161,2.913557e+12,437813398163,47491.59,12716.22,959


In [26]:
dim(official_dataset)

[1] 240  30

[1] 240  30

## Migration (flow) 2015-2020  
### Datasource: [Abel, Guy; E. Cohen, Joel (2019)](https://figshare.com/collections/Bilateral_international_migration_flow_estimates_for_200_countries/4470464)

This collection contains bilateral international migration flow estimates for 200 countries for five-year periods between 1990 and 2020. 

In [27]:
migration_flow <- read.csv("data/abel_cohen_migration_flow.csv")
migration_flow[is.na(migration_flow)] <- 0

head(migration_flow)

,year0,orig,dest,sd_drop_neg,sd_rev_neg,mig_rate,da_min_open,da_min_closed,da_pb_closed
,<int>,<chr>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>
1,1990,BDI,BDI,0,0,0,0,0,0
2,1990,COM,BDI,0,0,0,0,0,0
3,1990,DJI,BDI,0,0,0,0,0,0
4,1990,ERI,BDI,0,0,0,0,0,90
5,1990,ETH,BDI,0,0,0,0,0,2
6,1990,KEN,BDI,30,30,69,45,29,87


,year0,orig,dest,sd_drop_neg,sd_rev_neg,mig_rate,da_min_open,da_min_closed,da_pb_closed
,<int>,<chr>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>
1,1990,BDI,BDI,0,0,0,0,0,0
2,1990,COM,BDI,0,0,0,0,0,0
3,1990,DJI,BDI,0,0,0,0,0,0
4,1990,ERI,BDI,0,0,0,0,0,90
5,1990,ETH,BDI,0,0,0,0,0,2
6,1990,KEN,BDI,30,30,69,45,29,87


In [28]:
migration_flow <- migration_flow %>% 
                filter(year0 == 2015) %>%
                select(!year0) %>%
                rename(iso_d = dest, iso_o = orig, 
                       m_flow_sd_drop_neg = sd_drop_neg, 
                       m_flow_sd_rev_neg = sd_rev_neg, 
                       m_flow_mig_rate = mig_rate, 
                       m_flow_da_min_open = da_min_open, 
                       m_flow_da_min_closed = da_min_closed, 
                       m_flow_da_pb_closed = da_pb_closed)

head(migration_flow)

,iso_o,iso_d,m_flow_sd_drop_neg,m_flow_sd_rev_neg,m_flow_mig_rate,m_flow_da_min_open,m_flow_da_min_closed,m_flow_da_pb_closed
,<chr>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>
1,BDI,BDI,0,0,0,0,0,0
2,COM,BDI,0,0,0,0,0,0
3,DJI,BDI,0,0,0,0,0,0
4,ERI,BDI,0,131,0,43,60,183
5,ETH,BDI,0,14,0,9,3,24
6,KEN,BDI,194,194,211,1168,447,1554


,iso_o,iso_d,m_flow_sd_drop_neg,m_flow_sd_rev_neg,m_flow_mig_rate,m_flow_da_min_open,m_flow_da_min_closed,m_flow_da_pb_closed
,<chr>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>
1,BDI,BDI,0,0,0,0,0,0
2,COM,BDI,0,0,0,0,0,0
3,DJI,BDI,0,0,0,0,0,0
4,ERI,BDI,0,131,0,43,60,183
5,ETH,BDI,0,14,0,9,3,24
6,KEN,BDI,194,194,211,1168,447,1554


In [29]:
official_dataset <- merge(official_dataset, migration_flow)

head(official_dataset)

,iso_d,iso_o,destination,origin,col,csl,cnl,prox1,lp1,prox2,...,GDP_d,GDP_percapita_o,GDP_percapita_d,m_stock,m_flow_sd_drop_neg,m_flow_sd_rev_neg,m_flow_mig_rate,m_flow_da_min_open,m_flow_da_min_closed,m_flow_da_pb_closed
,<chr>,<chr>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,ARG,AUS,Argentina,Australia,0,0.02058416,0.00080000,0.25,1.945866,0.1305255,...,437813398163,58868.48,12716.22,279,1,1,56,16,105,2224
2,ARG,BRA,Argentina,Brazil,0,0.06052872,0.00080000,0.75,5.837599,0.4243976,...,437813398163,8592.21,12716.22,49647,475,475,9997,2332,0,10767
3,ARG,CHL,Argentina,Chile,1,0.98010004,0.85439998,0.00,0.000000,0.0000000,...,437813398163,13761.37,12716.22,216855,2079,2079,43666,13875,53604,84213
4,ARG,ESP,Argentina,Spain,1,0.98013985,0.85445821,0.00,0.000000,0.0000000,...,437813398163,28087.90,12716.22,99769,956,956,20090,6434,5516,49532
5,ARG,FRA,Argentina,France,0,0.13392781,0.01039232,0.50,3.891733,0.2545058,...,437813398163,38832.02,12716.22,8303,79,79,1672,467,156,3048
6,ARG,GBR,Argentina,Great Britain,0,0.08325152,0.00000000,0.25,1.945866,0.1305255,...,437813398163,47491.59,12716.22,959,8,8,193,45,126,1559


,iso_d,iso_o,destination,origin,col,csl,cnl,prox1,lp1,prox2,...,GDP_d,GDP_percapita_o,GDP_percapita_d,m_stock,m_flow_sd_drop_neg,m_flow_sd_rev_neg,m_flow_mig_rate,m_flow_da_min_open,m_flow_da_min_closed,m_flow_da_pb_closed
,<chr>,<chr>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,ARG,AUS,Argentina,Australia,0,0.02058416,0.00080000,0.25,1.945866,0.1305255,...,437813398163,58868.48,12716.22,279,1,1,56,16,105,2224
2,ARG,BRA,Argentina,Brazil,0,0.06052872,0.00080000,0.75,5.837599,0.4243976,...,437813398163,8592.21,12716.22,49647,475,475,9997,2332,0,10767
3,ARG,CHL,Argentina,Chile,1,0.98010004,0.85439998,0.00,0.000000,0.0000000,...,437813398163,13761.37,12716.22,216855,2079,2079,43666,13875,53604,84213
4,ARG,ESP,Argentina,Spain,1,0.98013985,0.85445821,0.00,0.000000,0.0000000,...,437813398163,28087.90,12716.22,99769,956,956,20090,6434,5516,49532
5,ARG,FRA,Argentina,France,0,0.13392781,0.01039232,0.50,3.891733,0.2545058,...,437813398163,38832.02,12716.22,8303,79,79,1672,467,156,3048
6,ARG,GBR,Argentina,Great Britain,0,0.08325152,0.00000000,0.25,1.945866,0.1305255,...,437813398163,47491.59,12716.22,959,8,8,193,45,126,1559


In [30]:
dim(official_dataset)

[1] 240  36

[1] 240  36

## WVS
### Datasource: [WVS (2020)](https://www.worldvaluessurvey.org/wvs.jsp)



In [31]:
wvs <- read.csv("data/subset-wvs-wave7-2020.csv")
wvs

country,S.SEV,T.SV
<chr>,<dbl>,<dbl>
Argentina,0.2276,-0.3849
Australia,2.2681,0.6354
Brazil,-0.0503,-0.3121
Chile,-0.0915,0.2214
France,1.4099,0.5362
Great Britain,2.3458,0.4881
Indonesia,-0.7675,-1.2452
Japan,1.2921,1.6625
Malaysia,-0.3557,0.0483


country,S.SEV,T.SV
<chr>,<dbl>,<dbl>
Argentina,0.2276,-0.3849
Australia,2.2681,0.6354
Brazil,-0.0503,-0.3121
Chile,-0.0915,0.2214
France,1.4099,0.5362
Great Britain,2.3458,0.4881
Indonesia,-0.7675,-1.2452
Japan,1.2921,1.6625
Malaysia,-0.3557,0.0483


In [32]:
#Cosine SIMILARITY
wvs_cosine <- as.data.frame(cosine(t(as.matrix(wvs[,2:3]))))
colnames(wvs_cosine) <- wvs$country
wvs_cosine["origin"] <- wvs$country


wvs_cosine_longer <- pivot_longer(wvs_cosine, -c("origin"), values_to = "CS_wvs", names_to = "destination")
head(wvs_cosine_longer)

origin,destination,CS_wvs
<chr>,<chr>,<dbl>
Argentina,Argentina,1.0000000
Argentina,Australia,0.2579213
Argentina,Brazil,0.7688173
Argentina,Chile,-0.9899189
Argentina,France,0.1697698
Argentina,Great Britain,0.3229716


origin,destination,CS_wvs
<chr>,<chr>,<dbl>
Argentina,Argentina,1.0000000
Argentina,Australia,0.2579213
Argentina,Brazil,0.7688173
Argentina,Chile,-0.9899189
Argentina,France,0.1697698
Argentina,Great Britain,0.3229716


In [33]:
wvs <- read.csv("data/subset-wvs-wave7-2020.csv")

#euclidean SIMILARITY
wvs_euc <- as.data.frame(1 - distance(as.matrix(wvs[,2:3]), method="euclidean")/max(distance(as.matrix(wvs[,2:3]), method="euclidean")))
colnames(wvs_euc) <- wvs$country
wvs_euc["origin"] <- wvs$country


wvs_euc_longer <- pivot_longer(wvs_euc, -c("origin"), values_to = "CS_euc_wvs", names_to = "destination")
head(wvs_euc_longer)

Metric: 'euclidean'; comparing: 16 vectors.

Metric: 'euclidean'; comparing: 16 vectors.

Metric: 'euclidean'; comparing: 16 vectors.

Metric: 'euclidean'; comparing: 16 vectors.



origin,destination,CS_euc_wvs
<chr>,<chr>,<dbl>
Argentina,Argentina,1.0000000
Argentina,Australia,0.3611266
Argentina,Brazil,0.9195511
Argentina,Chile,0.8081323
Argentina,France,0.5802905
Argentina,Great Britain,0.3584166


origin,destination,CS_euc_wvs
<chr>,<chr>,<dbl>
Argentina,Argentina,1.0000000
Argentina,Australia,0.3611266
Argentina,Brazil,0.9195511
Argentina,Chile,0.8081323
Argentina,France,0.5802905
Argentina,Great Britain,0.3584166


## Foursquare
### Datasource: [You Are What You Eat (and Drink): Identifying Cultural Boundaries by Analyzing Food and Drink Habits in Foursquare. Thiago H. Silva, Pedro O. S. Vaz de Melo, Jussara M. Almeida, Mirco Musolesi, Antonio A. F. Loureiro (2014)](https://www.aaai.org/ocs/index.php/ICWSM/ICWSM14/paper/viewPaper/8113)



In [34]:
foursquare <- read.csv("data/foursquare.csv")
foursquare

country,PC1,PC2
<chr>,<dbl>,<dbl>
Argentina,0.16,-0.03
Australia,0.05,0.17
Brazil,0.19,-0.18
Chile,0.17,-0.09
France,0.21,0.07
Great Britain,0.21,0.18
Indonesia,-0.28,-0.09
Japan,-0.20,0.09
Malaysia,-0.35,-0.19


country,PC1,PC2
<chr>,<dbl>,<dbl>
Argentina,0.16,-0.03
Australia,0.05,0.17
Brazil,0.19,-0.18
Chile,0.17,-0.09
France,0.21,0.07
Great Britain,0.21,0.18
Indonesia,-0.28,-0.09
Japan,-0.20,0.09
Malaysia,-0.35,-0.19


In [35]:
#cosine SIMILARITY
foursquare_cosine <- as.data.frame(cosine(t(as.matrix(foursquare[,2:3]))))
colnames(foursquare_cosine) <- foursquare$country
foursquare_cosine["origin"] <- foursquare$country


foursquare_cosine_longer <- pivot_longer(foursquare_cosine, -c("origin"), values_to = "CS_foursquare", names_to = "destination")
head(foursquare_cosine_longer)

origin,destination,CS_foursquare
<chr>,<chr>,<dbl>
Argentina,Argentina,1.0000000
Argentina,Australia,0.1005334
Argentina,Brazil,0.8402622
Argentina,Chile,0.9548769
Argentina,France,0.8741573
Argentina,Great Britain,0.6263188


origin,destination,CS_foursquare
<chr>,<chr>,<dbl>
Argentina,Argentina,1.0000000
Argentina,Australia,0.1005334
Argentina,Brazil,0.8402622
Argentina,Chile,0.9548769
Argentina,France,0.8741573
Argentina,Great Britain,0.6263188


In [36]:
foursquare <- read.csv("data/foursquare.csv")

#euclidean SIMILARITY
foursquare_euc <- as.data.frame(1 - distance(as.matrix(foursquare[,2:3]), method="euclidean")/max(distance(as.matrix(foursquare[,2:3]), method="euclidean")))
colnames(foursquare_euc) <- foursquare$country
foursquare_euc["origin"] <- foursquare$country


foursquare_euc_longer <- pivot_longer(foursquare_euc, -c("origin"), values_to = "CS_euc_foursquare", names_to = "destination")
head(foursquare_euc_longer)

Metric: 'euclidean'; comparing: 16 vectors.

Metric: 'euclidean'; comparing: 16 vectors.

Metric: 'euclidean'; comparing: 16 vectors.

Metric: 'euclidean'; comparing: 16 vectors.



origin,destination,CS_euc_foursquare
<chr>,<chr>,<dbl>
Argentina,Argentina,1.0000000
Argentina,Australia,0.6599275
Argentina,Brazil,0.7720915
Argentina,Chile,0.9093739
Argentina,France,0.8334258
Argentina,Great Britain,0.6783781


origin,destination,CS_euc_foursquare
<chr>,<chr>,<dbl>
Argentina,Argentina,1.0000000
Argentina,Australia,0.6599275
Argentina,Brazil,0.7720915
Argentina,Chile,0.9093739
Argentina,France,0.8334258
Argentina,Great Britain,0.6783781


In [37]:
official_dataset <- merge(official_dataset, wvs_cosine_longer)
official_dataset <- merge(official_dataset, wvs_euc_longer)
official_dataset <- merge(official_dataset, foursquare_cosine_longer)
official_dataset <- merge(official_dataset, foursquare_euc_longer)

head(official_dataset)

,destination,origin,iso_d,iso_o,col,csl,cnl,prox1,lp1,prox2,...,m_flow_sd_drop_neg,m_flow_sd_rev_neg,m_flow_mig_rate,m_flow_da_min_open,m_flow_da_min_closed,m_flow_da_pb_closed,CS_wvs,CS_euc_wvs,CS_foursquare,CS_euc_foursquare
,<chr>,<chr>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
1,Argentina,Australia,ARG,AUS,0,0.02058416,0.00080000,0.25,1.945866,0.13052547,...,1,1,56,16,105,2224,0.2579213,0.3611266,0.1005334,0.6599275
2,Argentina,Brazil,ARG,BRA,0,0.06052872,0.00080000,0.75,5.837599,0.42439765,...,475,475,9997,2332,0,10767,0.7688173,0.9195511,0.8402622,0.7720915
3,Argentina,Chile,ARG,CHL,1,0.98010004,0.85439998,0.00,0.000000,0.00000000,...,2079,2079,43666,13875,53604,84213,-0.9899189,0.8081323,0.9548769,0.9093739
4,Argentina,France,ARG,FRA,0,0.13392781,0.01039232,0.50,3.891733,0.25450581,...,79,79,1672,467,156,3048,0.1697698,0.5802905,0.8741573,0.8334258
5,Argentina,Great Britain,ARG,GBR,0,0.08325152,0.00000000,0.25,1.945866,0.13052547,...,8,8,193,45,126,1559,0.3229716,0.3584166,0.6263188,0.6783781
6,Argentina,Indonesia,ARG,IDN,0,0.00000000,0.00000000,0.00,0.000000,0.07436915,...,0,0,4,1,4,7,0.4656905,0.6316296,-0.8793284,0.3383839


,destination,origin,iso_d,iso_o,col,csl,cnl,prox1,lp1,prox2,...,m_flow_sd_drop_neg,m_flow_sd_rev_neg,m_flow_mig_rate,m_flow_da_min_open,m_flow_da_min_closed,m_flow_da_pb_closed,CS_wvs,CS_euc_wvs,CS_foursquare,CS_euc_foursquare
,<chr>,<chr>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
1,Argentina,Australia,ARG,AUS,0,0.02058416,0.00080000,0.25,1.945866,0.13052547,...,1,1,56,16,105,2224,0.2579213,0.3611266,0.1005334,0.6599275
2,Argentina,Brazil,ARG,BRA,0,0.06052872,0.00080000,0.75,5.837599,0.42439765,...,475,475,9997,2332,0,10767,0.7688173,0.9195511,0.8402622,0.7720915
3,Argentina,Chile,ARG,CHL,1,0.98010004,0.85439998,0.00,0.000000,0.00000000,...,2079,2079,43666,13875,53604,84213,-0.9899189,0.8081323,0.9548769,0.9093739
4,Argentina,France,ARG,FRA,0,0.13392781,0.01039232,0.50,3.891733,0.25450581,...,79,79,1672,467,156,3048,0.1697698,0.5802905,0.8741573,0.8334258
5,Argentina,Great Britain,ARG,GBR,0,0.08325152,0.00000000,0.25,1.945866,0.13052547,...,8,8,193,45,126,1559,0.3229716,0.3584166,0.6263188,0.6783781
6,Argentina,Indonesia,ARG,IDN,0,0.00000000,0.00000000,0.00,0.000000,0.07436915,...,0,0,4,1,4,7,0.4656905,0.6316296,-0.8793284,0.3383839


In [38]:
dim(official_dataset)

[1] 240  40

[1] 240  40

In [39]:
write.csv(official_dataset, "data/subset-official-dataset.csv", row.names = FALSE)

## Cultural Similarity 

#### symmetric:   

each country is represented by a vector corresponding to the top (almost) <b>400 dishes</b> (top 50 dishes in all the countries)

![CD](figs/heatmap-similarity-symmetric-tops-topK-cosine-reds.png)

In [40]:
cd_sym <- read.csv("data/FB-food-cultural_similarity-symmetric.csv", check.names=FALSE)
colnames(cd_sym)[1] <- "interests_from"

In [41]:
cd_sym

interests_from,Argentina,Australia,Brazil,Chile,France,Great Britain,Indonesia,Japan,Malaysia,Mexico,Russia,Singapore,South Korea,Spain,Turkey,United States
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Argentina,1.00000000,-0.1143361,0.176917369,0.39391861,-0.061345090,-0.1691530891,-0.19920772,-0.224371502,-0.28997372,0.27235146,-0.08055443,-0.2096671516,-0.20142284,0.163591513,-0.09868350,-0.12580614
Australia,-0.11433611,1.0000000,-0.183166299,-0.21693946,-0.181528839,0.3272786581,-0.42124163,-0.369408331,0.18000864,-0.19258554,-0.48793018,0.2374988871,-0.24214879,-0.213543938,-0.27776537,0.45026896
Brazil,0.17691737,-0.1831663,1.000000000,0.12196555,-0.009614683,-0.0914496937,0.01228442,0.040389545,-0.27893278,-0.02383473,0.10240044,-0.2598915766,-0.08731596,0.072308406,-0.03172250,-0.19253817
Chile,0.39391861,-0.2169395,0.121965552,1.00000000,-0.036574753,-0.1427083922,-0.12930396,-0.085777299,-0.32750818,0.34784223,-0.01501271,-0.2553625404,-0.20503456,0.194458741,-0.07814615,-0.14241346
France,-0.06134509,-0.1815288,-0.009614683,-0.03657475,1.000000000,-0.0164551200,0.02915838,0.097488102,-0.29143959,-0.10030008,0.24466473,-0.2547247320,-0.12086619,0.232377125,0.13634094,-0.14248141
Great Britain,-0.16915309,0.3272787,-0.091449694,-0.14270839,-0.016455120,1.0000000000,-0.23383791,-0.197735569,-0.06866661,-0.20550659,-0.16012196,0.0005845629,-0.15262230,-0.037024527,-0.16732636,0.23801647
Indonesia,-0.19920772,-0.4212416,0.012284416,-0.12930396,0.029158377,-0.2338379149,1.00000000,0.364092614,-0.02878002,-0.15926308,0.53888907,-0.2344010923,0.16082520,-0.051447082,0.20489202,-0.31442787
Japan,-0.22437150,-0.3694083,0.040389545,-0.08577730,0.097488102,-0.1977355688,0.36409261,1.000000000,-0.26822899,-0.17332967,0.47569199,-0.1261969938,0.21536567,-0.006820021,0.20586957,-0.29936526
Malaysia,-0.28997372,0.1800086,-0.278932779,-0.32750818,-0.291439588,-0.0686666064,-0.02878002,-0.268228986,1.00000000,-0.20696652,-0.44061942,0.5102216287,0.01868495,-0.348760866,-0.21755477,0.03649500


interests_from,Argentina,Australia,Brazil,Chile,France,Great Britain,Indonesia,Japan,Malaysia,Mexico,Russia,Singapore,South Korea,Spain,Turkey,United States
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Argentina,1.00000000,-0.1143361,0.176917369,0.39391861,-0.061345090,-0.1691530891,-0.19920772,-0.224371502,-0.28997372,0.27235146,-0.08055443,-0.2096671516,-0.20142284,0.163591513,-0.09868350,-0.12580614
Australia,-0.11433611,1.0000000,-0.183166299,-0.21693946,-0.181528839,0.3272786581,-0.42124163,-0.369408331,0.18000864,-0.19258554,-0.48793018,0.2374988871,-0.24214879,-0.213543938,-0.27776537,0.45026896
Brazil,0.17691737,-0.1831663,1.000000000,0.12196555,-0.009614683,-0.0914496937,0.01228442,0.040389545,-0.27893278,-0.02383473,0.10240044,-0.2598915766,-0.08731596,0.072308406,-0.03172250,-0.19253817
Chile,0.39391861,-0.2169395,0.121965552,1.00000000,-0.036574753,-0.1427083922,-0.12930396,-0.085777299,-0.32750818,0.34784223,-0.01501271,-0.2553625404,-0.20503456,0.194458741,-0.07814615,-0.14241346
France,-0.06134509,-0.1815288,-0.009614683,-0.03657475,1.000000000,-0.0164551200,0.02915838,0.097488102,-0.29143959,-0.10030008,0.24466473,-0.2547247320,-0.12086619,0.232377125,0.13634094,-0.14248141
Great Britain,-0.16915309,0.3272787,-0.091449694,-0.14270839,-0.016455120,1.0000000000,-0.23383791,-0.197735569,-0.06866661,-0.20550659,-0.16012196,0.0005845629,-0.15262230,-0.037024527,-0.16732636,0.23801647
Indonesia,-0.19920772,-0.4212416,0.012284416,-0.12930396,0.029158377,-0.2338379149,1.00000000,0.364092614,-0.02878002,-0.15926308,0.53888907,-0.2344010923,0.16082520,-0.051447082,0.20489202,-0.31442787
Japan,-0.22437150,-0.3694083,0.040389545,-0.08577730,0.097488102,-0.1977355688,0.36409261,1.000000000,-0.26822899,-0.17332967,0.47569199,-0.1261969938,0.21536567,-0.006820021,0.20586957,-0.29936526
Malaysia,-0.28997372,0.1800086,-0.278932779,-0.32750818,-0.291439588,-0.0686666064,-0.02878002,-0.268228986,1.00000000,-0.20696652,-0.44061942,0.5102216287,0.01868495,-0.348760866,-0.21755477,0.03649500


In [42]:
cd_sym <- cd_sym %>% pivot_longer(!interests_from, names_to = "origin", values_to = "CS_symm") %>%
                        rename(destination = interests_from)

head(cd_sym)

destination,origin,CS_symm
<chr>,<chr>,<dbl>
Argentina,Argentina,1.00000000
Argentina,Australia,-0.11433611
Argentina,Brazil,0.17691737
Argentina,Chile,0.39391861
Argentina,France,-0.06134509
Argentina,Great Britain,-0.16915309


destination,origin,CS_symm
<chr>,<chr>,<dbl>
Argentina,Argentina,1.00000000
Argentina,Australia,-0.11433611
Argentina,Brazil,0.17691737
Argentina,Chile,0.39391861
Argentina,France,-0.06134509
Argentina,Great Britain,-0.16915309


In [43]:
dim(cd_sym)

[1] 256   3

[1] 256   3

#### nonsymmetric:   
each country is represented by a vector corresponding to the <b>top 50 dishes from origin country</b>

![CDiv](figs/heatmap-similarity-topK-cosine-reds.png)

The **(non-symmetric)** Cultural Similarity between **origin (rows) and destination (columns)** represent how similar origin and destination are in terms of popular dishes from the country of **origin**.   

**Hypothesis:** Immigrants prefer to move to (host) countries culturally similar to their (home/previous) country.

In [44]:
cd_nonsym <- read.csv("data/FB-food-cultural_similarity-nonsymmetric.csv", check.names=FALSE)
colnames(cd_nonsym)[1] <- "interests_from"

In [45]:
cd_food_o <- cd_nonsym %>% pivot_longer(!interests_from, names_to = "destination", values_to = "CS_nonsymm_food_o") %>%
                           rename(origin = interests_from)
                           
head(cd_food_o)

origin,destination,CS_nonsymm_food_o
<chr>,<chr>,<dbl>
Argentina,Argentina,1.0000000
Argentina,Australia,0.3185665
Argentina,Brazil,0.1077964
Argentina,Chile,0.4667300
Argentina,France,-0.4108677
Argentina,Great Britain,-0.2072795


origin,destination,CS_nonsymm_food_o
<chr>,<chr>,<dbl>
Argentina,Argentina,1.0000000
Argentina,Australia,0.3185665
Argentina,Brazil,0.1077964
Argentina,Chile,0.4667300
Argentina,France,-0.4108677
Argentina,Great Britain,-0.2072795


In [46]:
dim(cd_food_o)

[1] 256   3

[1] 256   3

The **(non-symmetric)** Cultural Similarity between **origin (rows) and destination (columns)** representing how similar origin and destination are in terms of popular dishes from the country of **destination**. 

In [47]:
cd_food_d <- cd_nonsym %>% pivot_longer(!interests_from, names_to = "origin", values_to = "CS_nonsymm_food_d") %>%
                            rename(destination = interests_from)

head(cd_food_d)

destination,origin,CS_nonsymm_food_d
<chr>,<chr>,<dbl>
Argentina,Argentina,1.0000000
Argentina,Australia,0.3185665
Argentina,Brazil,0.1077964
Argentina,Chile,0.4667300
Argentina,France,-0.4108677
Argentina,Great Britain,-0.2072795


destination,origin,CS_nonsymm_food_d
<chr>,<chr>,<dbl>
Argentina,Argentina,1.0000000
Argentina,Australia,0.3185665
Argentina,Brazil,0.1077964
Argentina,Chile,0.4667300
Argentina,France,-0.4108677
Argentina,Great Britain,-0.2072795


In [48]:
dim(cd_food_d)

[1] 256   3

[1] 256   3

In [49]:
cd <- merge(cd_food_o, cd_food_d)
cd <- merge(cd_sym, cd)

cd

destination,origin,CS_symm,CS_nonsymm_food_o,CS_nonsymm_food_d
<chr>,<chr>,<dbl>,<dbl>,<dbl>
Argentina,Argentina,1.00000000,1.00000000,1.00000000
Argentina,Australia,-0.11433611,-0.25534750,0.31856652
Argentina,Brazil,0.17691737,0.21724563,0.10779643
Argentina,Chile,0.39391861,0.40901942,0.46672998
Argentina,France,-0.06134509,-0.21927129,-0.41086766
Argentina,Great Britain,-0.16915309,-0.36688149,-0.20727953
Argentina,Indonesia,-0.19920772,-0.31759939,-0.67796209
Argentina,Japan,-0.22437150,-0.51795720,-0.81593583
Argentina,Malaysia,-0.28997372,-0.82887197,-0.26388265


destination,origin,CS_symm,CS_nonsymm_food_o,CS_nonsymm_food_d
<chr>,<chr>,<dbl>,<dbl>,<dbl>
Argentina,Argentina,1.00000000,1.00000000,1.00000000
Argentina,Australia,-0.11433611,-0.25534750,0.31856652
Argentina,Brazil,0.17691737,0.21724563,0.10779643
Argentina,Chile,0.39391861,0.40901942,0.46672998
Argentina,France,-0.06134509,-0.21927129,-0.41086766
Argentina,Great Britain,-0.16915309,-0.36688149,-0.20727953
Argentina,Indonesia,-0.19920772,-0.31759939,-0.67796209
Argentina,Japan,-0.22437150,-0.51795720,-0.81593583
Argentina,Malaysia,-0.28997372,-0.82887197,-0.26388265


In [50]:
dim(cd)

[1] 256   5

[1] 256   5

#### Dataset: official statistics + FB cultural distance

In [51]:
dataset <- merge(official_dataset, cd)#, mutate(official_dataset, destination = tolower(destination), origin = tolower(origin)))

head(dataset)

,destination,origin,iso_d,iso_o,col,csl,cnl,prox1,lp1,prox2,...,m_flow_da_min_open,m_flow_da_min_closed,m_flow_da_pb_closed,CS_wvs,CS_euc_wvs,CS_foursquare,CS_euc_foursquare,CS_symm,CS_nonsymm_food_o,CS_nonsymm_food_d
,<chr>,<chr>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,Argentina,Australia,ARG,AUS,0,0.02058416,0.00080000,0.25,1.945866,0.13052547,...,16,105,2224,0.2579213,0.3611266,0.1005334,0.6599275,-0.11433611,-0.2553475,0.3185665
2,Argentina,Brazil,ARG,BRA,0,0.06052872,0.00080000,0.75,5.837599,0.42439765,...,2332,0,10767,0.7688173,0.9195511,0.8402622,0.7720915,0.17691737,0.2172456,0.1077964
3,Argentina,Chile,ARG,CHL,1,0.98010004,0.85439998,0.00,0.000000,0.00000000,...,13875,53604,84213,-0.9899189,0.8081323,0.9548769,0.9093739,0.39391861,0.4090194,0.4667300
4,Argentina,France,ARG,FRA,0,0.13392781,0.01039232,0.50,3.891733,0.25450581,...,467,156,3048,0.1697698,0.5802905,0.8741573,0.8334258,-0.06134509,-0.2192713,-0.4108677
5,Argentina,Great Britain,ARG,GBR,0,0.08325152,0.00000000,0.25,1.945866,0.13052547,...,45,126,1559,0.3229716,0.3584166,0.6263188,0.6783781,-0.16915309,-0.3668815,-0.2072795
6,Argentina,Indonesia,ARG,IDN,0,0.00000000,0.00000000,0.00,0.000000,0.07436915,...,1,4,7,0.4656905,0.6316296,-0.8793284,0.3383839,-0.19920772,-0.3175994,-0.6779621


,destination,origin,iso_d,iso_o,col,csl,cnl,prox1,lp1,prox2,...,m_flow_da_min_open,m_flow_da_min_closed,m_flow_da_pb_closed,CS_wvs,CS_euc_wvs,CS_foursquare,CS_euc_foursquare,CS_symm,CS_nonsymm_food_o,CS_nonsymm_food_d
,<chr>,<chr>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,Argentina,Australia,ARG,AUS,0,0.02058416,0.00080000,0.25,1.945866,0.13052547,...,16,105,2224,0.2579213,0.3611266,0.1005334,0.6599275,-0.11433611,-0.2553475,0.3185665
2,Argentina,Brazil,ARG,BRA,0,0.06052872,0.00080000,0.75,5.837599,0.42439765,...,2332,0,10767,0.7688173,0.9195511,0.8402622,0.7720915,0.17691737,0.2172456,0.1077964
3,Argentina,Chile,ARG,CHL,1,0.98010004,0.85439998,0.00,0.000000,0.00000000,...,13875,53604,84213,-0.9899189,0.8081323,0.9548769,0.9093739,0.39391861,0.4090194,0.4667300
4,Argentina,France,ARG,FRA,0,0.13392781,0.01039232,0.50,3.891733,0.25450581,...,467,156,3048,0.1697698,0.5802905,0.8741573,0.8334258,-0.06134509,-0.2192713,-0.4108677
5,Argentina,Great Britain,ARG,GBR,0,0.08325152,0.00000000,0.25,1.945866,0.13052547,...,45,126,1559,0.3229716,0.3584166,0.6263188,0.6783781,-0.16915309,-0.3668815,-0.2072795
6,Argentina,Indonesia,ARG,IDN,0,0.00000000,0.00000000,0.00,0.000000,0.07436915,...,1,4,7,0.4656905,0.6316296,-0.8793284,0.3383839,-0.19920772,-0.3175994,-0.6779621


In [52]:
dim(dataset)

[1] 240  43

[1] 240  43

In [53]:
dataset <- dummy_cols(dataset, select_columns = 'origin')
dataset <- dummy_cols(dataset, select_columns = 'destination')

In [54]:
dim(dataset)

[1] 240  75

[1] 240  75

In [55]:
write.csv(dataset, "data/subset-final-dataset.csv", row.names = FALSE)

In [56]:
dataset <- read.csv("data/subset-final-dataset.csv")

dataset

destination,origin,iso_d,iso_o,col,csl,cnl,prox1,lp1,prox2,...,destination_Indonesia,destination_Japan,destination_Malaysia,destination_Mexico,destination_Russia,destination_Singapore,destination_South.Korea,destination_Spain,destination_Turkey,destination_United.States
<chr>,<chr>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
Argentina,Australia,ARG,AUS,0,0.02058416,0.00080000,0.25000000,1.9458665,0.13052547,...,0,0,0,0,0,0,0,0,0,0
Argentina,Brazil,ARG,BRA,0,0.06052872,0.00080000,0.75000000,5.8375993,0.42439765,...,0,0,0,0,0,0,0,0,0,0
Argentina,Chile,ARG,CHL,1,0.98010004,0.85439998,0.00000000,0.0000000,0.00000000,...,0,0,0,0,0,0,0,0,0,0
Argentina,France,ARG,FRA,0,0.13392781,0.01039232,0.50000000,3.8917329,0.25450581,...,0,0,0,0,0,0,0,0,0,0
Argentina,Great Britain,ARG,GBR,0,0.08325152,0.00000000,0.25000000,1.9458665,0.13052547,...,0,0,0,0,0,0,0,0,0,0
Argentina,Indonesia,ARG,IDN,0,0.00000000,0.00000000,0.00000000,0.0000000,0.07436915,...,0,0,0,0,0,0,0,0,0,0
Argentina,Japan,ARG,JPN,0,0.00000000,0.00000000,0.00000000,0.0000000,0.06896222,...,0,0,0,0,0,0,0,0,0,0
Argentina,Malaysia,ARG,MYS,0,0.00000000,0.00000000,0.00000000,0.0000000,0.08409883,...,0,0,0,0,0,0,0,0,0,0
Argentina,Mexico,ARG,MEX,1,0.98010004,0.88319999,0.00000000,0.0000000,0.00000000,...,0,0,0,0,0,0,0,0,0,0


destination,origin,iso_d,iso_o,col,csl,cnl,prox1,lp1,prox2,...,destination_Indonesia,destination_Japan,destination_Malaysia,destination_Mexico,destination_Russia,destination_Singapore,destination_South.Korea,destination_Spain,destination_Turkey,destination_United.States
<chr>,<chr>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
Argentina,Australia,ARG,AUS,0,0.02058416,0.00080000,0.25000000,1.9458665,0.13052547,...,0,0,0,0,0,0,0,0,0,0
Argentina,Brazil,ARG,BRA,0,0.06052872,0.00080000,0.75000000,5.8375993,0.42439765,...,0,0,0,0,0,0,0,0,0,0
Argentina,Chile,ARG,CHL,1,0.98010004,0.85439998,0.00000000,0.0000000,0.00000000,...,0,0,0,0,0,0,0,0,0,0
Argentina,France,ARG,FRA,0,0.13392781,0.01039232,0.50000000,3.8917329,0.25450581,...,0,0,0,0,0,0,0,0,0,0
Argentina,Great Britain,ARG,GBR,0,0.08325152,0.00000000,0.25000000,1.9458665,0.13052547,...,0,0,0,0,0,0,0,0,0,0
Argentina,Indonesia,ARG,IDN,0,0.00000000,0.00000000,0.00000000,0.0000000,0.07436915,...,0,0,0,0,0,0,0,0,0,0
Argentina,Japan,ARG,JPN,0,0.00000000,0.00000000,0.00000000,0.0000000,0.06896222,...,0,0,0,0,0,0,0,0,0,0
Argentina,Malaysia,ARG,MYS,0,0.00000000,0.00000000,0.00000000,0.0000000,0.08409883,...,0,0,0,0,0,0,0,0,0,0
Argentina,Mexico,ARG,MEX,1,0.98010004,0.88319999,0.00000000,0.0000000,0.00000000,...,0,0,0,0,0,0,0,0,0,0


### LOG 10 some variables

**Migration stock/flow** 

In order to calculate `log10()` of my dependent variable, *`m_stock`*, I will add an **offset (`offset=1`)** to all the observations in the *`m_stock`* column.

In [57]:
dataset$m_stock <- dataset$m_stock + 1.0
dataset$m_flow_sd_drop_neg <- dataset$m_flow_sd_drop_neg + 1.0
dataset$m_flow_sd_rev_neg <- dataset$m_flow_sd_rev_neg + 1.0
dataset$m_flow_mig_rate <- dataset$m_flow_mig_rate + 1.0
dataset$m_flow_da_min_open <- dataset$m_flow_da_min_open + 1.0
dataset$m_flow_da_min_closed <- dataset$m_flow_da_min_closed + 1.0
dataset$m_flow_da_pb_closed <- dataset$m_flow_da_pb_closed + 1.0

head(dataset)

,destination,origin,iso_d,iso_o,col,csl,cnl,prox1,lp1,prox2,...,destination_Indonesia,destination_Japan,destination_Malaysia,destination_Mexico,destination_Russia,destination_Singapore,destination_South.Korea,destination_Spain,destination_Turkey,destination_United.States
,<chr>,<chr>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,Argentina,Australia,ARG,AUS,0,0.02058416,0.00080000,0.25,1.945866,0.13052547,...,0,0,0,0,0,0,0,0,0,0
2,Argentina,Brazil,ARG,BRA,0,0.06052872,0.00080000,0.75,5.837599,0.42439765,...,0,0,0,0,0,0,0,0,0,0
3,Argentina,Chile,ARG,CHL,1,0.98010004,0.85439998,0.00,0.000000,0.00000000,...,0,0,0,0,0,0,0,0,0,0
4,Argentina,France,ARG,FRA,0,0.13392781,0.01039232,0.50,3.891733,0.25450581,...,0,0,0,0,0,0,0,0,0,0
5,Argentina,Great Britain,ARG,GBR,0,0.08325152,0.00000000,0.25,1.945866,0.13052547,...,0,0,0,0,0,0,0,0,0,0
6,Argentina,Indonesia,ARG,IDN,0,0.00000000,0.00000000,0.00,0.000000,0.07436915,...,0,0,0,0,0,0,0,0,0,0


,destination,origin,iso_d,iso_o,col,csl,cnl,prox1,lp1,prox2,...,destination_Indonesia,destination_Japan,destination_Malaysia,destination_Mexico,destination_Russia,destination_Singapore,destination_South.Korea,destination_Spain,destination_Turkey,destination_United.States
,<chr>,<chr>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,Argentina,Australia,ARG,AUS,0,0.02058416,0.00080000,0.25,1.945866,0.13052547,...,0,0,0,0,0,0,0,0,0,0
2,Argentina,Brazil,ARG,BRA,0,0.06052872,0.00080000,0.75,5.837599,0.42439765,...,0,0,0,0,0,0,0,0,0,0
3,Argentina,Chile,ARG,CHL,1,0.98010004,0.85439998,0.00,0.000000,0.00000000,...,0,0,0,0,0,0,0,0,0,0
4,Argentina,France,ARG,FRA,0,0.13392781,0.01039232,0.50,3.891733,0.25450581,...,0,0,0,0,0,0,0,0,0,0
5,Argentina,Great Britain,ARG,GBR,0,0.08325152,0.00000000,0.25,1.945866,0.13052547,...,0,0,0,0,0,0,0,0,0,0
6,Argentina,Indonesia,ARG,IDN,0,0.00000000,0.00000000,0.00,0.000000,0.07436915,...,0,0,0,0,0,0,0,0,0,0


In [58]:
dataset$log10_m_flow_da_pb_closed <- log10(dataset$m_flow_da_pb_closed)
dataset$log10_m_stock <- log10(dataset$m_stock)
dataset$log10_pop_o <- log10(dataset$pop_o)
dataset$log10_pop_d <- log10(dataset$pop_d)
dataset$log10_area_o <- log10(dataset$area_o)
dataset$log10_area_d <- log10(dataset$area_d)
dataset$log10_GDP_o <- log10(dataset$GDP_o)
dataset$log10_GDP_d <- log10(dataset$GDP_d)
dataset$log10_GDP_percapita_o <- log10(dataset$GDP_percapita_o)
dataset$log10_GDP_percapita_d <- log10(dataset$GDP_percapita_d)
dataset$log10_distwces <- log10(dataset$distwces)
dataset$log10_distcap <- log10(dataset$distcap)

head(dataset)

,destination,origin,iso_d,iso_o,col,csl,cnl,prox1,lp1,prox2,...,log10_pop_o,log10_pop_d,log10_area_o,log10_area_d,log10_GDP_o,log10_GDP_d,log10_GDP_percapita_o,log10_GDP_percapita_d,log10_distwces,log10_distcap
,<chr>,<chr>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,Argentina,Australia,ARG,AUS,0,0.02058416,0.00080000,0.25,1.945866,0.13052547,...,7.401456,7.651091,6.885748,6.441992,12.16152,11.64129,4.769883,4.104358,4.079842,4.069442
2,Argentina,Brazil,ARG,BRA,0,0.06052872,0.00080000,0.75,5.837599,0.42439765,...,8.324384,7.651091,6.930028,6.441992,12.37373,11.64129,3.934105,4.104358,3.319997,3.371669
3,Argentina,Chile,ARG,CHL,1,0.98010004,0.85439998,0.00,0.000000,0.00000000,...,7.277656,7.651091,5.879064,6.441992,11.45490,11.64129,4.138662,4.104358,2.973884,3.052431
4,Argentina,France,ARG,FRA,0,0.13392781,0.01039232,0.50,3.891733,0.25450581,...,7.813779,7.651091,5.738008,6.441992,12.47304,11.64129,4.589190,4.104358,4.038297,4.044236
5,Argentina,Great Britain,ARG,GBR,0,0.08325152,0.00000000,0.25,1.945866,0.13052547,...,7.829498,7.651091,5.387586,6.441992,12.46442,11.64129,4.676617,4.104358,4.046426,4.047177
6,Argentina,Indonesia,ARG,IDN,0,0.00000000,0.00000000,0.00,0.000000,0.07436915,...,8.432369,7.651091,6.286380,6.441992,12.08079,11.64129,3.590220,4.104358,4.192234,4.182865


,destination,origin,iso_d,iso_o,col,csl,cnl,prox1,lp1,prox2,...,log10_pop_o,log10_pop_d,log10_area_o,log10_area_d,log10_GDP_o,log10_GDP_d,log10_GDP_percapita_o,log10_GDP_percapita_d,log10_distwces,log10_distcap
,<chr>,<chr>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,Argentina,Australia,ARG,AUS,0,0.02058416,0.00080000,0.25,1.945866,0.13052547,...,7.401456,7.651091,6.885748,6.441992,12.16152,11.64129,4.769883,4.104358,4.079842,4.069442
2,Argentina,Brazil,ARG,BRA,0,0.06052872,0.00080000,0.75,5.837599,0.42439765,...,8.324384,7.651091,6.930028,6.441992,12.37373,11.64129,3.934105,4.104358,3.319997,3.371669
3,Argentina,Chile,ARG,CHL,1,0.98010004,0.85439998,0.00,0.000000,0.00000000,...,7.277656,7.651091,5.879064,6.441992,11.45490,11.64129,4.138662,4.104358,2.973884,3.052431
4,Argentina,France,ARG,FRA,0,0.13392781,0.01039232,0.50,3.891733,0.25450581,...,7.813779,7.651091,5.738008,6.441992,12.47304,11.64129,4.589190,4.104358,4.038297,4.044236
5,Argentina,Great Britain,ARG,GBR,0,0.08325152,0.00000000,0.25,1.945866,0.13052547,...,7.829498,7.651091,5.387586,6.441992,12.46442,11.64129,4.676617,4.104358,4.046426,4.047177
6,Argentina,Indonesia,ARG,IDN,0,0.00000000,0.00000000,0.00,0.000000,0.07436915,...,8.432369,7.651091,6.286380,6.441992,12.08079,11.64129,3.590220,4.104358,4.192234,4.182865


In [59]:
countries

[1] "Argentina"     "Australia"     "Brazil"        "Chile"        
 [5] "France"        "Great Britain" "Indonesia"     "Japan"        
 [9] "Malaysia"      "Mexico"        "Russia"        "Singapore"    
[13] "South Korea"   "Spain"         "Turkey"        "United States"

[1] "Argentina"     "Australia"     "Brazil"        "Chile"        
 [5] "France"        "Great Britain" "Indonesia"     "Japan"        
 [9] "Malaysia"      "Mexico"        "Russia"        "Singapore"    
[13] "South Korea"   "Spain"         "Turkey"        "United States"

In [60]:
write.csv(dataset, "data/subset-final-dataset.csv", row.names = FALSE)